**Criando conexão no sistema.**

In [ ]:
from google.cloud.dataproc_spark_connect import DataprocSparkSession
from google.cloud.dataproc_v1 import Session
from pyspark.sql import functions as F

# This will create a default Spark session
spark = DataprocSparkSession.builder.getOrCreate()


/usr/local/lib/python3.12/dist-packages/google/cloud/dataproc_spark_connect/session.py:332: UserWarning: Python version mismatch detected: Client is using Python 3.12, but Dataproc runtime 3.0 uses Python 3.11. This mismatch may cause issues with Python UDF (User Defined Function) compatibility. Consider using Python 3.11 for optimal UDF execution.
  dataproc_config: Session = self._get_dataproc_config()


██████████████████████████████████████████████▍                                 

**Lendo tabela da Silver.**

In [ ]:
df = spark.read.format("bigquery").option("table", "clinic-de.2_silver.patients").load()

df.printSchema()

root
 |-- patient_id: string (nullable = true)
 |-- blood_type: string (nullable = true)
 |-- card_number: long (nullable = false)
 |-- plan_name: string (nullable = true)
 |-- has_plan: boolean (nullable = false)
 |-- phones: string (nullable = true)
 |-- email: string (nullable = true)
 |-- zip: string (nullable = true)
 |-- country: string (nullable = true)
 |-- state: string (nullable = true)
 |-- city: string (nullable = true)
 |-- street: string (nullable = true)
 |-- sex: string (nullable = true)
 |-- birth_date: date (nullable = true)
 |-- name: string (nullable = true)
 |-- lng: double (nullable = true)
 |-- lat: double (nullable = true)



**Criando nova coluna para identificar a idade dos pacientes e classificando de acordo.**

In [ ]:
df = df.withColumn(
    "age",
    F.floor(F.datediff(F.current_date(), F.col("birth_date")) / 365.25)
)

In [ ]:
df1 = df.withColumn(
    "age_band",
    F.when(F.col("age") < 17, "adolescente")
     .when((F.col("age") >= 17) & (F.col("age") <= 25), "jovem adulto")
     .when((F.col("age") > 25) & (F.col("age") <= 45), "adulto")
     .when((F.col("age") > 45) & (F.col("age") <= 60), "meia idade")
     .when(F.col("age") > 60, "idoso")
     .otherwise(None)
)

**Salvando dataframe na Gold.**

In [ ]:
(df1.write
        .format("bigquery")
        .option("table", "clinic-de.3_gold.dim_patients")
        .option("writeMethod", "direct")
        .mode("overwrite")
        .save()
    )

  0%|           0/1 Tasks

In [ ]:
spark.stop()